In [1]:
import jax.numpy as jnp
from jax import vmap

from jaxgp.utils import CovMatrixDD, CovMatrixFD, CovMatrixFF
from jaxgp.kernels import RBF

In [2]:
x1 = jnp.zeros((5,1))
x2 = jnp.zeros((5,2))
x3 = jnp.zeros((5,3))
x4 = jnp.zeros((5,4))

params = jnp.array((1.0, 1.0))
kernel = RBF()

In [3]:
ff1d = CovMatrixFF(x1, x1, kernel,params)
ff2d = CovMatrixFF(x2, x2, kernel,params)
ff3d = CovMatrixFF(x3, x3, kernel,params)
ff4d = CovMatrixFF(x4, x4, kernel,params)

print(ff1d.shape, ff2d.shape, ff3d.shape, ff4d.shape)

(5, 5) (5, 5) (5, 5) (5, 5)


In [4]:
fd1d = CovMatrixFD(x1, x1, kernel,params)
fd2d = CovMatrixFD(x2, x2, kernel,params)
fd3d = CovMatrixFD(x3, x3, kernel,params)
fd4d = CovMatrixFD(x4, x4, kernel,params)

print(fd1d.shape, fd2d.shape, fd3d.shape, fd4d.shape)

fd1d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x1, x1, kernel,params))
fd2d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x2, x2, kernel,params))
fd3d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x3, x3, kernel,params))
fd4d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x4, x4, kernel,params))

print(fd1d.shape, fd2d.shape, fd3d.shape, fd4d.shape)

(5, 5) (5, 10) (5, 15) (5, 20)
(5, 5) (5, 10) (5, 15) (5, 20)


In [5]:
dd1d = CovMatrixDD(x1, x1, kernel,params)
dd2d = CovMatrixDD(x2, x2, kernel,params)
dd3d = CovMatrixDD(x3, x3, kernel,params)
dd4d = CovMatrixDD(x4, x4, kernel,params)

print(dd1d.shape, dd2d.shape, dd3d.shape, dd4d.shape)

dd1d = jnp.hstack(jnp.hstack((*CovMatrixDD(x1, x1, kernel, params),)))
dd2d = jnp.hstack(jnp.hstack((*CovMatrixDD(x2, x2, kernel, params),)))
dd3d = jnp.hstack(jnp.hstack((*CovMatrixDD(x3, x3, kernel, params),)))
dd4d = jnp.hstack(jnp.hstack((*CovMatrixDD(x4, x4, kernel, params),)))

print(dd1d.shape, dd2d.shape, dd3d.shape, dd4d.shape)

(5, 5) (10, 10) (15, 15) (20, 20)
(25,) (100,) (225,) (400,)


In [6]:
from jaxgp.predict import full_gradient_predict

In [7]:
covmat = jnp.ones((10,10))
X_split = (jnp.ones((1,2)), jnp.ones((9,2)))
Y = jnp.ones(10)

X = jnp.ones((20,2))

In [8]:
first = CovMatrixFD(X_split[0], X, kernel, params)
second = CovMatrixDD(X_split[1], X, kernel, params)

In [9]:
print(first.shape, second.shape)

(1, 40) (18, 40)


## Creating the training data set

`jax.numpy` has almost the same usage as the standard `numpy` package, with the caveat that `jax.ndarray` is an immutable type, meaning that no inplace changes can be made. For creating training data this should however not be an issue.

In [10]:
import jax.numpy as jnp
from jax import random

We will model a simple sin function:

In [11]:
# true function is a noisy lennard jones potential
def sin(x, noise=0.0, key = random.PRNGKey(0)):
    return jnp.sin(x) + random.normal(key,x.shape, dtype=jnp.float32)*noise

def cos(x, noise=0.0, key = random.PRNGKey(0)):
    return jnp.cos(x) + random.normal(key,x.shape, dtype=jnp.float32)*noise

To define the training data we first need to define boundaries to choose the datapoints from. Then, random points are chosen in this interval. `random.split` creates a new subkey from the previous key to get a new sudo-random sample from the uniform distribution.

In [12]:
# Interval bounds from which to choose the data points
bounds = jnp.array([0.0, 2*jnp.pi])

# How many function and derivative observations should be chosen
num_f_vals = (1,)
num_d_vals = (10,)

# initial seed for the pseudo random key generation
seed = 0

# create new keys and randomly sample the above interval for training features
key, subkey = random.split(random.PRNGKey(seed))
x_func = random.uniform(subkey, num_f_vals, minval=bounds[0], maxval=bounds[1])
key, subkey = random.split(key)
x_der = random.uniform(subkey, num_d_vals, minval=bounds[0], maxval=bounds[1])

# noise with which to sample the training labels
noise = 0.1
key, subkey = random.split(key)
y_func = sin(x_func,noise, subkey)
key, subkey = random.split(key)
y_der = cos(x_der, noise, subkey)

The GPR framework needs as input for training a tuple of arrays `X_split` of which contains a set of points where the function is sampled and a set of points where the gradient is sampled. Both array in `X_split` is of shape `(n_samples_i, N)`. `X_split` should be ordered as follows: the first array represents the datapoints for the function observations and the second array represents the gradient of the function. `Y_train` should just be an array of shape `(n_samples_function + n_samples_gradient,)`.

In [13]:
# reshaping needs to be done the get the arrays in the form (n_samples_i, N)
X_split = [x_func.reshape(-1,1),x_der.reshape(-1,1)]
print(y_der.shape)
print(y_func.shape)
print(jnp.hstack((y_func, y_der)).shape)
Y_train = (y_func, y_der) # jnp.hstack((y_func, y_der))

(10,)
(1,)
(11,)


In [14]:
print(x_func, x_der)

[0.6620528] [2.9177153 5.8826876 2.1381183 4.9896216 5.3221016 4.163432  5.31646
 3.161748  4.623604  6.050371 ]


### Defining the Kernel and its initial parameters

The kernels can be found in `jaxgp.kernels`. Currently implemented are `RBF`, `Linear`, and `Periodic` kernels. When in doubt what kernel to use, go with an `RBF` kernel.

In [15]:
from jaxgp.kernels import RBF

kernel = RBF()
# an RBF kernel has per default 2 parameters
init_kernel_params = jnp.array([2.0, 2.0])

In [16]:
from jaxgp.bayesopt import BayesOpt, upper_confidence_bound

bounds = jnp.array([[0.0],[2*jnp.pi]])
eval_func = lambda x: (cos(x, noise, key), True)
extra_args = (bounds, eval_func)

model = BayesOpt(X_split, Y_train, kernel, init_kernel_params, noise, upper_confidence_bound, extra_args)

In [17]:
model.run(10)

[[1.7063252]] [[-0.03824922]]


TypeError: Cannot concatenate arrays with shapes that differ in dimensions other than the one being concatenated: concatenating along dimension 1 for shapes (10, 1), (1, 1).